In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, results as rs

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2993429379104) was called 1 times


2021-01-05 09:12:09,155 - NumExpr defaulting to 8 threads.


In [5]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2993429379968) was called 1 times


In [7]:
fa,unmapped = feat.preprocess_faults(scs_raw,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2993429378960) was called 1 times
Running: Function "load_tote_lookup" (id=2993429380688) was called 1 times
Running: Function "load_module_lookup" (id=2993426479856) was called 1 times
Running: Function "load_ID_lookup" (id=2993429377376) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:294: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i] = fa_floor['Time Left'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i+1] = fa_floor['Duration'][i+1] - fa_floor['Duration'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [9]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av)

Running: Function "load_PTT_lookup" (id=2993429379392) was called 1 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2993429378816) was called 1 times
Running: Function "load_PTT_lookup" (id=2993429379392) was called 2 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2993429378816) was called 2 times
Running: Function "load_PTT_lookup" (id=2993429379392) was called 3 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2993429378816) was called 3 times
Running: Function "load_PTT_lookup" (id=2993429379392) was called 4 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2993429378816) was called 4 times
Running: Function "load_PTT_lookup" (id=2993429379392) was called 5 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2993429378816) was called 5 times
Running: Function "load_PTT_lookup" (id=2993429379392) was called 6 times
Previous Hours Weighted
Running

In [10]:
X,y = md.gen_feat_var(df,target = 'Availability', features = ['Totes','Faults'])
y=1-y

In [11]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})


R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)



Cross Validation Scores: 
 
        R2 Scores
1       0.461081
2    -171.483301
3       0.494509
4       0.489704
5       0.386784
Mean  -33.930244
STD    68.776539
292
                                 OLS Regression Results                                
Dep. Variable:           Availability   R-squared (uncentered):                   0.844
Model:                            OLS   Adj. R-squared (uncentered):              0.840
Method:                 Least Squares   F-statistic:                              217.7
Date:                Tue, 05 Jan 2021   Prob (F-statistic):                        0.00
Time:                        09:17:14   Log-Likelihood:                          4772.3
No. Observations:               11899   AIC:                                     -8967.
Df Residuals:                   11610   BIC:                                     -6832.
Df Model:                         289                                                  
Covariance Type:            nonrobust 

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [12]:
R2_OOS

-2.157892845829758

In [13]:
Output = rs.create_output(fa_PTT,Coeff)

In [14]:
Output

,Alert ID,Alert,Fault ID,Asset Code,Tote Colour,Quadrant,MODULE,Original_timestamp,PTT011,PTT012,...,PTT151,PTT152,PTT171,PTT172,PTT181,PTT182,PTT191,PTT192,PTT201,PTT202
0,24714.0,C05: C0503RDC254 254B12 occupancy sensor fault,3416.0,C0503RDC254,Blue,1.0,0,2020-11-20 03:21:39,789.374104,789.374104,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,116944.0,C07: C0701STA020 020B12 occupancy sensor fault,3416.0,C0701STA020,Grey,1.0,0,2020-12-01 08:26:41,448.171196,448.171196,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,121537.0,C06: C0601ACH018 018B11 block sensor fault,3420.0,C0601ACH018,Grey,1.0,0,2020-12-06 21:46:17,262.916914,262.916914,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,121537.0,C06: C0601ACH018 018B11 block sensor fault,3420.0,C0601ACH018,Grey,1.0,0,2020-12-06 22:00:00,733.164420,733.164420,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,26002.0,C23: C2303RDC082 082B12 occupancy sensor fault,3416.0,C2303RDC082,Blue,0.0,0,2020-11-20 00:12:12,239.008164,239.008164,...,239.008164,239.008164,239.008164,239.008164,239.008164,239.008164,239.008164,239.008164,239.008164,239.008164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7764,145749.0,SCS205: sto: timeout bin from lifter to car X:...,5105.0,SCS205,Grey,4.0,20,2020-12-09 21:51:24,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,73.951942,73.951942
7765,145485.0,SCS205: sto: timeout bin from lifter to car X:...,5105.0,SCS205,Grey,4.0,20,2020-12-09 22:24:06,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.396252,34.396252
7766,144984.0,SCS205: sto: timeout bin from lifter to car X:...,5105.0,SCS205,Grey,4.0,20,2020-12-09 23:42:45,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.452521,7.452521
7767,145097.0,SCS205: sto: timeout bin from lifter to car X:...,5105.0,SCS205,Grey,4.0,20,2020-12-09 23:20:35,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.608735,11.608735
